Условие: дана таблица с колонками (id, name, salary, managerId), студентам необходимо написать код на spark, который создаст эту таблицу (данные указаны ниже) и в результате выдаст таблицу в которой будут имена сотрудников, которые зарабатывают больше своих менеджеров.
Данные для таблицы:

data = {'id': [1, 2, 3, 4], 'Name': ['Joe', 'Henry', 'Sam', 'Max'],
        'Salary':['70', '80', '60', '90'], 'ManagerId':['3', '4', 'Null', 'Null']}
        

In [1]:
#Устанавливаем pySpark
!pip install pyspark >> None

In [73]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, avg, when, max, month
from pyspark.sql.functions import countDistinct
from pyspark.sql import functions as F
import pandas as pd
from pyspark.sql import Row

In [123]:
# Создание Spark сессии
spark = SparkSession.builder.appName("Employees").getOrCreate()

# Создание DataFrame
data = [(1, "Joe", "70", "3"),
        (2, "Henry", "80", "4"),
        (3, "Sam", "60", "None"),
        (4, "Max", "90", "None")]

df = spark.createDataFrame(data, ['id',	"Name",	"Salary", "ManagerId"])

# Преобразование типов
df = df.withColumn("Salary", df["Salary"].cast("int"))
df = df.withColumn("ManagerId", df["ManagerId"].cast("int"))

df.show()

+---+-----+------+---------+
| id| Name|Salary|ManagerId|
+---+-----+------+---------+
|  1|  Joe|    70|        3|
|  2|Henry|    80|        4|
|  3|  Sam|    60|     NULL|
|  4|  Max|    90|     NULL|
+---+-----+------+---------+



In [124]:
# разделяем таблицу на работников и менеджеров
emp_df = df.filter(df.ManagerId.isNotNull())
man_df = df.filter(df.ManagerId.isNull()).drop('ManagerId').withColumnRenamed('Salary', 'man_Salary').withColumnRenamed('Name', 'man_Name')
emp_df.show()
man_df.show()


+---+-----+------+---------+
| id| Name|Salary|ManagerId|
+---+-----+------+---------+
|  1|  Joe|    70|        3|
|  2|Henry|    80|        4|
+---+-----+------+---------+

+---+--------+----------+
| id|man_Name|man_Salary|
+---+--------+----------+
|  3|     Sam|        60|
|  4|     Max|        90|
+---+--------+----------+



In [128]:
# объединяем таблици
df = emp_df.join(man_df, emp_df.ManagerId == man_df.id)
df.show()

+---+-----+------+---------+---+--------+----------+
| id| Name|Salary|ManagerId| id|man_Name|man_Salary|
+---+-----+------+---------+---+--------+----------+
|  1|  Joe|    70|        3|  3|     Sam|        60|
|  2|Henry|    80|        4|  4|     Max|        90|
+---+-----+------+---------+---+--------+----------+



In [129]:
# отфильтровываем результат
res = df.select(df['Name']).filter(df.Salary > df.man_Salary)
res.show()

+----+
|Name|
+----+
| Joe|
+----+

